In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
import io

In [11]:
data_string = '''Names Estimate Std.Error
(Intercept) 3446.830     81.840
totwrk        -0.169      0.018
age            2.714      1.472
south        102.272     41.925
male          87.108     35.173
smsa         -54.188     33.193
yngkid       -13.051     50.459
marr          31.360     42.263
union         11.866     38.186'''

In [12]:
df = pd.read_csv(io.StringIO(data_string), sep='\s+')
df

,Names,Estimate,Std.Error
0,(Intercept),3446.830,81.840
1,totwrk,-0.169,0.018
2,age,2.714,1.472
3,south,102.272,41.925
4,male,87.108,35.173
5,smsa,-54.188,33.193
6,yngkid,-13.051,50.459
7,marr,31.360,42.263
8,union,11.866,38.186


In [13]:
df['znachimost'] = np.abs(df['Estimate']/df['Std.Error'])
df

,Names,Estimate,Std.Error,znachimost
0,(Intercept),3446.830,81.840,42.116691
1,totwrk,-0.169,0.018,9.388889
2,age,2.714,1.472,1.843750
3,south,102.272,41.925,2.439404
4,male,87.108,35.173,2.476559
5,smsa,-54.188,33.193,1.632513
6,yngkid,-13.051,50.459,0.258646
7,marr,31.360,42.263,0.742020
8,union,11.866,38.186,0.310742


In [14]:
alpha = 0.05
#ppf - percent point function
t = stats.t.ppf(1-alpha/2, 707-9)
t

1.9633684547051231

In [15]:
df['result'] = df.apply(lambda x: 'значимо' if x['znachimost']>t else 'не значимо',axis=1)
df

,Names,Estimate,Std.Error,znachimost,result
0,(Intercept),3446.830,81.840,42.116691,значимо
1,totwrk,-0.169,0.018,9.388889,значимо
2,age,2.714,1.472,1.843750,не значимо
3,south,102.272,41.925,2.439404,значимо
4,male,87.108,35.173,2.476559,значимо
5,smsa,-54.188,33.193,1.632513,не значимо
6,yngkid,-13.051,50.459,0.258646,не значимо
7,marr,31.360,42.263,0.742020,не значимо
8,union,11.866,38.186,0.310742,не значимо


In [17]:
df_labour = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
df_labour

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709
...,...,...,...,...
564,2.625403,20,1.424376,33.477545
565,1.276386,61,2.109048,26.300732
566,1.953869,117,6.241870,41.153979
567,1.318527,46,7.902237,66.720139


In [22]:
model = smf.ols(data=df_labour, formula='np.log(output)~np.log(capital)+np.log(labour)').fit()
model.summary(alpha=0.01)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     1462.
Date:                Wed, 22 Mar 2023   Prob (F-statistic):          2.64e-224
Time:                        09:48:44   Log-Likelihood:                -385.95
No. Observations:                 569   AIC:                             777.9
Df Residuals:                     566   BIC:                             790.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.005      0.995]
-----------------------------------------------------------------------------------
Intercept          -1.7115      0.097    -17.697      0.000      -1.961      -1.462
np.log(capital)     0.2076      0.017     12.077      0.000       0.163       0.252
np.log(labour)      0.7148      0.023     30.890      0.000       0.655       0.775
==============================================================================
Omnibus:                      259.475   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2677.293
Skew:                           1.734   Prob(JB):                         0.00
Kurtosis:                      13.045   Cond. No.                         24.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
model.summary(alpha=0.01).tables[1]

,coef,std err,t,P>|t|,[0.005,0.995]
Intercept,-1.7115,0.097,-17.697,0.000,-1.961,-1.462
np.log(capital),0.2076,0.017,12.077,0.000,0.163,0.252
np.log(labour),0.7148,0.023,30.890,0.000,0.655,0.775


In [26]:
model.t_test('Intercept')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -1.7115      0.097    -17.697      0.000      -1.901      -1.522

In [28]:
model.t_test('np.log(capital)=0.5')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.2076      0.017    -17.014      0.000       0.174       0.241

In [31]:
len(df_labour)

569

In [33]:
t_cr = stats.t.ppf(1-0.01/2, len(df_labour)-3)
t_cr

2.584543428450176

### Критическое значение Стьюдента меньше наблюдаемого, коэф np.log(capital) значительно отличается от 0.2076

In [35]:
#подсчет t набл вручную
beta_cap = 0.2076
st_err = 0.017
t_nab = np.abs((beta_cap-0.5)/st_err)
t_nab

17.2